<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Optimization_project3_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Script that creates the table for the recomendation system**
Run this File before the Recomendation System file

In [1]:
### TITLE RATINGS & TITLE EPISODE 

## TITLE RATINGS - Contains the IMDb rating and votes information for titles.
# tconst (string) - alphanumeric unique identifier of the title
# averageRating – weighted average of all the individual user ratings
# numVotes - number of votes the title has received

In [2]:
import pandas as pd
import seaborn as sns
import pickle

In [3]:
col_list = ['tconst','primaryTitle', 'startYear','genres','titleType']
title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list)

In [4]:
col_list2 = ['tconst','averageRating','numVotes']
title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list2)

In [5]:
t_b_r = pd.merge(title_basics, title_ratings, how='outer', on=['tconst', 'tconst'])

In [6]:
# Tables with just movies and shorts with the start year of 2022 or older.

condition1 = t_b_r['titleType'] == 'movie'
condition2 = t_b_r['titleType'] == 'short'
condition3 = t_b_r['startYear'] <= '2022'
m_s = t_b_r[condition1 | condition2 & condition3 ][['tconst','titleType','primaryTitle','startYear','genres','averageRating','numVotes']]

In [7]:
# add the wheighted_IMDB column ( we took the formula from the imdb)

C = m_s['averageRating'].mean()
m_s['wheighted_IMDB'] = ((m_s['numVotes']/(m_s['numVotes']+25000))*m_s['averageRating']) + ((25000/(m_s['numVotes']+25000))*C)

In [8]:
# Split and explode so we have just one genre per row

m_s["genres"] = m_s.genres.str.split(',')
m_s_split = m_s.explode("genres")

## Pre-processing

In [9]:
# slice df to not contain \\N's
condition = m_s_split.loc[m_s_split['startYear'] != '\\N']

In [10]:
#drop averageRating, numVotes
condition.drop(['averageRating', 'numVotes'], axis = 1, inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
#Genre get dummies
condition = pd.concat([condition , condition['genres'].str.get_dummies()], 
          axis = 1)

In [12]:
condition

,tconst,titleType,primaryTitle,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,Animation,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,tt0000001,short,Carmencita,1894,Documentary,6.354083,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,tt0000001,short,Carmencita,1894,Short,6.354083,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,tt0000002,short,Le clown et ses chiens,1892,Animation,6.398172,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,tt0000002,short,Le clown et ses chiens,1892,Short,6.398172,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,tt0000003,short,Pauvre Pierrot,1892,Animation,6.410575,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9374760,tt9916730,movie,6 Gunn,2017,\N,6.404879,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9374770,tt9916754,movie,Chico Albuquerque - Revelações,2013,Documentary,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9374771,tt9916756,short,Pretty Pretty Black Girl,2019,Short,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9374775,tt9916764,short,38,2018,Short,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
#do a groupby because we had to explode for the genre
condition_g = condition.groupby('primaryTitle').max().sort_values(by = 'wheighted_IMDB', ascending = False)

In [14]:
#convert year to int to use in the model
condition_g['startYear'].astype('int')

primaryTitle
The Shawshank Redemption                             1994
The Godfather                                        2013
The Dark Knight                                      2020
The Lord of the Rings: The Return of the King        2003
Schindler's List                                     1993
                                                     ... 
überRICH                                             2017
Čáp - Moments of Decisions                           2016
Śakra                                                2023
Šentilj-Spielfeld - Border Crossing That Once Was    2009
Τhe Improvisation of Petros Mokas                    2022
Name: startYear, Length: 1097440, dtype: int64

In [15]:
condition_gi = condition_g.reset_index().reset_index()
condition_gi.head(10)

,index,primaryTitle,tconst,titleType,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,0,The Shawshank Redemption,tt0111161,movie,1994,Drama,9.273049,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,The Godfather,tt5429154,short,2013,Short,9.162608,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,The Dark Knight,tt14607756,short,2020,Short,8.975596,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,The Lord of the Rings: The Return of the King,tt0167260,movie,2003,Drama,8.965110,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Schindler's List,tt0108052,movie,1993,History,8.952729,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,The Godfather Part II,tt0071562,movie,1974,Drama,8.949632,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,12 Angry Men,tt0050083,movie,1957,Drama,8.919964,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,Pulp Fiction,tt0110912,movie,1994,Drama,8.869740,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,Inception,tt7321322,short,2014,Short,8.774612,1,0,1,...,0,0,1,1,0,0,0,0,0,0
9,9,Fight Club,tt16225632,short,2022,Short,8.771900,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
condition_gi.to_pickle('./condition_gi.pickle')